<a href="https://colab.research.google.com/github/malinphy/recommender_sys/blob/main/convolutional_recommender_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet d2l==0.17.0 ;
!pip install --quiet -U mxnet-cu101==1.7.0 ;
!pip install --quiet tensorflow-datasets ;
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import random 
import random
import mxnet as mx
from mxnet import gluon, np, npx
from mxnet.gluon import nn
from d2l import mxnet as d2l
# import tensorflow_datasets as tfds

import tensorflow as tf 
from tensorflow import keras 
# from tensorflow.keras import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.layers import Dropout,Lambda

In [2]:
TARGET_NUM = 1
batch_size = 4096
L=5
d=16
d_prime=4 
drop_ratio=0.05
num_factors= 10
num_users =943
num_items =1682

In [3]:
class SeqDataset(gluon.data.Dataset):
    def __init__(self, user_ids, item_ids, L, num_users, num_items,
                 candidates):
        user_ids, item_ids = np.array(user_ids), np.array(item_ids)
        sort_idx = np.array(
            sorted(range(len(user_ids)), key=lambda k: user_ids[k]))
        u_ids, i_ids = user_ids[sort_idx], item_ids[sort_idx]
        temp, u_ids, self.cand = {}, u_ids.asnumpy(), candidates
        self.all_items = set([i for i in range(num_items)])
        [temp.setdefault(u_ids[i], []).append(i) for i, _ in enumerate(u_ids)]
        temp = sorted(temp.items(), key=lambda x: x[0])
        u_ids = np.array([i[0] for i in temp])
        idx = np.array([i[1][0] for i in temp])
        self.ns = ns = int(
            sum([
                c - L if c >= L + 1 else 1
                for c in np.array([len(i[1]) for i in temp])]))
        self.seq_items = np.zeros((ns, L))
        self.seq_users = np.zeros(ns, dtype='int32')
        self.seq_tgt = np.zeros((ns, 1))
        self.test_seq = np.zeros((num_users, L))
        test_users, _uid = np.empty(num_users), None
        for i, (uid, i_seq) in enumerate(self._seq(u_ids, i_ids, idx, L + 1)):
            if uid != _uid:
                self.test_seq[uid][:] = i_seq[-L:]
                test_users[uid], _uid = uid, uid
            self.seq_tgt[i][:] = i_seq[-1:]
            self.seq_items[i][:], self.seq_users[i] = i_seq[:L], uid

    def _win(self, tensor, window_size, step_size=1):
        if len(tensor) - window_size >= 0:
            for i in range(len(tensor), 0, -step_size):
                if i - window_size >= 0:
                    yield tensor[i - window_size:i]
                else:
                    break
        else:
            yield tensor

    def _seq(self, u_ids, i_ids, idx, max_len):
        for i in range(len(idx)):
            stop_idx = None if i >= len(idx) - 1 else int(idx[i + 1])
            for s in self._win(i_ids[int(idx[i]):stop_idx], max_len):
                yield (int(u_ids[i]), s)

    def __len__(self):
        return self.ns

    def __getitem__(self, idx):
        neg = list(self.all_items - set(self.cand[int(self.seq_users[idx])]))
        i = random.randint(0, len(neg) - 1)
        return (self.seq_users[idx], self.seq_items[idx], self.seq_tgt[idx],
                neg[i])

In [4]:
TARGET_NUM, L, batch_size = 1, 5, 4096
df, num_users, num_items = d2l.read_data_ml100k()
train_data, test_data = d2l.split_data_ml100k(df, num_users, num_items,
                                              'seq-aware')
users_train, items_train, ratings_train, candidates = d2l.load_data_ml100k(
    train_data, num_users, num_items, feedback="implicit")
users_test, items_test, ratings_test, test_iter = d2l.load_data_ml100k(
    test_data, num_users, num_items, feedback="implicit")
train_seq_data = SeqDataset(users_train, items_train, L, num_users, num_items,
                            candidates)
train_iter = gluon.data.DataLoader(train_seq_data, batch_size, True,
                                   last_batch="rollover",
                                   num_workers=d2l.get_dataloader_workers())
test_seq_iter = train_seq_data.test_seq
train_seq_data[0]


(array([0], dtype=int32),
 array([241., 170., 110., 255.,   4.]),
 array([101.]),
 504)

In [5]:
pos_1 = []
pos_2 = []
pos_3 = []
pos_4 = []
for i in train_seq_data:
  pos_1.append(int(i[0]) )
  pos_2.append((i[1]) )
  pos_3.append(int(i[2]))
  pos_4.append(int(i[3]))
# I believe, in this part seq variable or input is obvious,
# output is single number in the array

def expander(x):
    return (tf.reshape(x,shape= [x.shape[0],x.shape[1],x.shape[2],1], name=None))
    
def seqer(x):
    return (tf.squeeze(x, axis=2))


In [6]:
big_arr = []
for i in range(len(pos_2)):
  x = pos_2[i]
  small_arr = []
  for j in x:
    # print(int(j))
    small_arr.append(int(j))
  big_arr.append(small_arr)

In [7]:
np.array(big_arr, dtype= 'int32')

array([[ 241,  170,  110,  255,    4],
       [ 188,  241,  170,  110,  255],
       [  31,  188,  241,  170,  110],
       ...,
       [  27,  173,   99, 1066,  126],
       [ 180,   27,  173,   99, 1066],
       [  63,  180,   27,  173,   99]], dtype=int32)

In [52]:
class Caser(nn.Block):
    def __init__(self, num_factors, num_users, num_items, L=5, d=16,
                 d_prime=4, drop_ratio=0.05, **kwargs):
        super(Caser, self).__init__(**kwargs)
        self.P = Embedding(num_users, num_factors)
        self.Q = Embedding(num_items, num_factors)
        self.d_prime, self.d = d_prime, d
        # Vertical convolution layer
        self.conv_v = nn.Conv2D(d_prime, (L, 1), in_channels=1)
        # Horizontal convolution layer
        h = [i + 1 for i in range(L)]
        self.conv_h, self.max_pool = nn.Sequential(), nn.Sequential()
        for i in h:
            self.conv_h.add(nn.Conv2D(d, (i, num_factors), in_channels=1))
            self.max_pool.add(nn.MaxPool1D(L - i + 1))
        # Fully-connected layer
        self.fc1_dim_v, self.fc1_dim_h = d_prime * num_factors, d * len(h)
        self.fc = nn.Dense(in_units=d_prime * num_factors + d * L,
                           activation='relu', units=num_factors)
        self.Q_prime = nn.Embedding(num_items, num_factors * 2)
        self.b = nn.Embedding(num_items, 1)
        self.dropout = nn.Dropout(drop_ratio)

    def forward(self, user_id, seq, item_id):
        h = [i + 1 for i in range(L)]
        # item_embs = np.expand_dims(self.Q(seq), 1)
        item_embs = self.Q(seq)
        user_emb = self.P(user_id)
        out, out_h, out_v, out_hs = None, None, None, []
        if self.d_prime:
            # out_v = self.conv_v(item_embs)
            out_v = Conv2D(d_prime, (L, 1))(expander(item_embs))
            out_v = out_v.reshape(out_v.shape[0], self.fc1_dim_v)

        # if self.d_prime:
        #     out_v = self.conv_v(item_embs)
        #     out_v = out_v.reshape(out_v.shape[0], self.fc1_dim_v)
        if self.d:
            
            for i in h:
              linear_model = tf.keras.Sequential([
              Lambda(expander),
              Conv2D(d, (i, num_factors), activation = 'relu'),
              Lambda(seqer),
              MaxPool1D(L-i+1)
              ])
              out_hs.append(linear_model(item_embs))
            out_h = tf.concat(out_hs, axis=1)
        out_h = tf.reshape(out_h, [out_h.shape[0], out_h.shape[1]*out_h.shape[2] ])
        out = np.concatenate([out_v, out_h], axis=1)
        z = self.fc(self.dropout(out))
        x = np.concatenate([z, user_emb], axis=1)
        q_prime_i = np.squeeze(self.Q_prime(item_id))
        b = np.squeeze(self.b(item_id))
        res = (x * q_prime_i).sum(1) + b
        return res


In [53]:
user_id = tf.keras.Input(shape= (), dtype = 'float64')
seq = tf.keras.Input(shape= (), dtype = 'float64')
item_id = tf.keras.Input(shape= (), dtype = 'float64')

In [56]:
# targets = Caser(num_factors = 10,
#       num_users = 943, 
#       num_items = 1682, 
#       L=5, 
#       d=16,
#       d_prime=4, 
#       drop_ratio=0.05)(user_id,seq, item_id)

# # caser_model = tf.keras.Model(inputs = [user_id,seq,item_id], outputs = res)

In [ ]:
caser_model.compile(
    optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999,
    # epsilon=1e-07,
    # amsgrad=False,
    # name='Adam'), 
    ),
    loss=tf.keras.losses.Hinge(), 
    # loss = 
    metrics=["accuracy"]
)

In [ ]:
caser_model.fit(([tf.constant(pos_1),tf.constant(big_arr),
                  tf.constant(pos_3)]),
                 tf.constant(pos_4),
                epochs =1,
                # batch_size = 5
                )

In [ ]:
  model = tf.keras.Sequential([
     tf.keras.layers.Dense(3, activation = "relu", name = "firstLayer"),
     tf.keras.layers.Dense(4, activation = "tanh", name = "secondLayer"),
     tf.keras.layers.Dense(2, name = "lastLayer"),
  ])

  # Create a input and pass it to model
  # input = tf.random.normal((3, 4))
  # output = model(input)

In [ ]:
print(model.weights)

In [ ]:
deneme = tf.constant([[1,2,3],[1,2,3]])
deneme

In [ ]:
deneme = (deneme[tf.newaxis,:])

In [ ]:
deneme

In [ ]:
letters = ['a', 'b', 'c']
numbers = [0, 1, 2]
for l, n in zip(letters, numbers):
     print(f'Letter: {l}',f'Number: {n}')
    #  print(f'Number: {n}')
